In [103]:
path= "abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/"
orders = spark.read.option("header", "true") .csv(path + "orders")





StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 105, Finished, Available)

In [104]:
from pyspark.sql.functions import col, broadcast,when

StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 106, Finished, Available)

# Load data for mapping

In [105]:
dim_delivery_status = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_delivery_status")


StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 107, Finished, Available)

In [106]:
dim_delivery_risk = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_delivery_risk LIMIT 1000")


StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 108, Finished, Available)

In [107]:
dim_order_status = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_order_status LIMIT 1000")


StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 109, Finished, Available)

In [108]:
dim_shipping_mode = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_shipping_mode LIMIT 1000")

StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 110, Finished, Available)

In [109]:
dim_transaction = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_transaction LIMIT 1000")

StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 111, Finished, Available)

In [110]:
dim_territory = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_territory")


StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 112, Finished, Available)

In [111]:
dim_store = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_store")


StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 113, Finished, Available)

In [112]:
dim_customer = spark.sql("SELECT customer_key, customer_id FROM DQT_Gold_LakeHouse_Layer.dim_customer ")


StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 114, Finished, Available)

In [113]:
dim_product = spark.sql("SELECT product_key,product_card_id FROM DQT_Gold_LakeHouse_Layer.dim_product")


StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 115, Finished, Available)

# Mapping for orders

## order_status_key

In [114]:
dim_order_status = broadcast(dim_order_status)
orders = orders.withColumnRenamed("order_status", "order_status_t1")

orders = orders.join(dim_order_status, orders['order_status_t1'] == dim_order_status['order_status'],"inner")

orders = orders.drop("order_status_t1", "order_status")

StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 116, Finished, Available)

## delivery_status_key

In [115]:
# Broadcast dim_delivery_status DataFrame
dim_delivery_status = broadcast(dim_delivery_status)

#renamed
orders = orders.withColumnRenamed("delivery_status", "delivery_status_t1")

# Join orders with dim_delivery_status
orders = orders.join(dim_delivery_status, orders['delivery_status_t1'] == dim_delivery_status['delivery_status'],"inner")


orders = orders.drop("delivery_status", "delivery_status_t1")

StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 117, Finished, Available)

## transaction_key

In [116]:
# Broadcast dim_transaction DataFrame
dim_transaction = broadcast(dim_transaction)

#renamed
orders = orders.withColumnRenamed("type_of_transaction", "type_of_transaction_t1")

# Join orders with dim_transaction
orders = orders.join(dim_transaction, orders['type_of_transaction_t1'] == dim_transaction['type_of_transaction'],"inner")

orders = orders.drop("type_of_transaction", "type_of_transaction_t1")



StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 118, Finished, Available)

## shipping_mode_key

In [117]:
# Broadcast dim_dim_shipping_mode DataFrame
dim_shipping_mode = broadcast(dim_shipping_mode)

#renamed
orders = orders.withColumnRenamed("shipping_mode", "shipping_mode_t1")

# Join orders with dim_dim_shipping_mode
orders = orders.join(dim_shipping_mode, orders['shipping_mode_t1'] == dim_shipping_mode['shipping_mode'],"inner")

orders = orders.drop("shipping_mode", "shipping_mode_t1")


StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 119, Finished, Available)

## customer

In [118]:
orders = orders.withColumnRenamed("customer_id", "customer_id_t1")
orders = orders.join(dim_customer, dim_customer['customer_id'] == orders['customer_id_t1'], "inner")
orders = orders.drop("customer_id_t1", "customer_id")



StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 120, Finished, Available)

## product

In [119]:
orders = orders.withColumnRenamed("product_card_id", "product_card_id_t1")
orders = orders.join(dim_product , dim_product.product_card_id == orders.product_card_id_t1, "inner")
orders = orders.drop("product_card_id", "product_card_id_t1")




StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 121, Finished, Available)

## dim_territory

In [120]:


# Join orders with dim_dim_shipping_mode
orders = orders.join(dim_territory, orders['concat_territory'] == dim_territory['ter_concat_territory'],"inner")

orders = orders.drop("concat_territory", "ter_concat_territory","market", "order_region", "order_state", "order_country", "order_city")


StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 122, Finished, Available)

## dim_store_region

In [121]:


# Join orders with dim_dim_shipping_mode
orders = orders.join(dim_store, orders['concat_store_address'] == dim_store['st_concat_store_address'],"inner")

orders = orders.drop("st_concat_store_address", "concat_store_address","store_country", "store_city","store_street", "longitude","latitude" )



StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 123, Finished, Available)

## format date orders and shipping

In [122]:
from pyspark.sql.functions import to_timestamp, date_format

StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 124, Finished, Available)

In [123]:
orders = orders.withColumn("order_time_key", 
date_format(to_timestamp("order_date_dateorders", "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"), "HHmm")
)

StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 125, Finished, Available)

In [124]:
orders = orders.withColumn("shipping_time_key",

date_format(to_timestamp("shipping_date_dateorders", "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"), "HHmm")
)

StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 126, Finished, Available)

In [125]:
orders = orders.withColumn("shipping_date_dateorders", 
                   date_format(to_timestamp("shipping_date_dateorders", "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"), "yyyyMMdd"))
orders = orders.withColumnRenamed("shipping_date_dateorders", "shipping_date_key")


StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 127, Finished, Available)

In [126]:
orders = orders.withColumn("order_date_dateorders", 
                   date_format(to_timestamp("order_date_dateorders", "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"), "yyyyMMdd"))
orders = orders.withColumnRenamed("order_date_dateorders", "order_date_key")



StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 128, Finished, Available)

## format_late_delivery_risk

In [127]:
orders = orders.withColumn("late_delivery_risk",when(col("late_delivery_risk") == '0', '2').otherwise('1'))
orders = orders.withColumnRenamed("late_delivery_risk","delivery_risk_key")


StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 129, Finished, Available)

# create_fact


In [128]:
orders = orders.drop(*['year', 'month'])
orders.createOrReplaceTempView("orders")

StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 130, Finished, Available)

In [129]:
from pyspark.sql.types import * 
from pyspark.sql.functions import * 

StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 131, Finished, Available)

## fact_sales

In [130]:
# Cast columns to appropriate data types


fact_sales = spark.sql("""

SELECT order_id, product_key, order_date_key, shipping_date_key, delivery_risk_key, sales, 
order_status_key, delivery_status_key, transaction_key, shipping_mode_key, territory_key,
store_key, order_time_key, shipping_time_key, customer_key,
benefit_per_order, order_item_discount, order_item_quantity FROM orders

""")


StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 132, Finished, Available)

In [131]:
fact_sales= fact_sales.select(
    col("order_id").cast("string"),
    col("product_key").cast("int"),
    col("order_date_key").cast("int"),
    col("order_time_key").cast("int"),
    col("shipping_date_key").cast("int"),
    col("shipping_time_key").cast("int"),
    col("delivery_risk_key").cast("int"),
    col("customer_key").cast("int"),
    col("sales").cast("float"),
    col("order_status_key").cast("int"),
    col("delivery_status_key").cast("int"),
    col("transaction_key").cast("int"),
    col("shipping_mode_key").cast("int"),
    col("territory_key").cast("int"),
    col("store_key").cast("int"),
    col("benefit_per_order").cast("float"),
    col("order_item_discount").cast("float"),
    col("order_item_quantity").cast("int")
)


StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 133, Finished, Available)

## fact_monthly_kpi

In [133]:
fact_monthly_kpi = spark.sql("""
  WITH  delivery_status_4_cte AS (
        SELECT 
            dd.year, 
            dd.month, 
            o.store_key, 
            COUNT(delivery_status_key) AS on_time_delivery
        FROM 
            (SELECT 
                order_id, delivery_status_key, order_date_key, store_key 
            FROM 
                orders 
            GROUP BY 
                order_id, delivery_status_key, order_date_key, store_key
            ) AS o
        JOIN 
            dim_date dd 
        ON 
            o.order_date_key = dd.date_key
        WHERE 
            delivery_status_key = 4 
        GROUP BY 
            dd.year, dd.month, o.store_key
    ),

    total_cte AS (
        SELECT 
            dd.year, 
            dd.month, 
            o.store_key, 
            COUNT(o.delivery_status_key) AS total_delivery_status,
            SUM(o.total_sales) AS actual_sales,
            SUM(o.total_profit) AS actual_profit
        FROM 
            (SELECT 
                order_id, 
                delivery_status_key, 
                customer_key, 
                order_date_key, 
                store_key, 
                SUM(sales) AS total_sales ,
                SUM(benefit_per_order) AS total_profit
            FROM 
                orders 
            GROUP BY 
                order_id, 
                delivery_status_key, 
                order_date_key, 
                customer_key, 
                store_key
            ) AS o 
        JOIN 
            dim_date dd ON o.order_date_key = dd.date_key 
        GROUP BY 
            dd.year, dd.month, o.store_key
    ),
    total_cus AS (
            SELECT 
                dd.year,
                dd.month,
                store_key,
                COUNT(DISTINCT o.customer_key) AS total_customer
            FROM 
                orders o
            JOIN 
                dim_date dd ON dd.date_key = o.order_date_key
            GROUP BY 
                dd.year,dd.month, store_key
    ),

    cus_cte AS (
            SELECT 
                year, 
                month, 
                store_key, 
                SUM(CASE WHEN num_orders_in_month >= 2 THEN 1 ELSE 0 END) as num_cus_return
            FROM (
                SELECT 
                    dd.year,
                    dd.month,
                    o.customer_key,
                    o.store_key,
                    COUNT(DISTINCT o.order_id) AS num_orders_in_month
                FROM 
                    dim_date dd
                RIGHT JOIN 
                    orders o ON dd.date_key = o.order_date_key AND o.order_date_key <= (SELECT MAX(order_date_key) FROM orders)
                GROUP BY 
                    dd.year, dd.month, o.customer_key, o.store_key
            ) as subquery
            GROUP BY 
                year, month, store_key
    )

SELECT 
    date_format(last_day(to_date(concat(ds.year, '-', ds.month, '-', '01'))), 'yyyyMMdd') AS date_key,
    ds.store_key, 
    ds.on_time_delivery,
    tc.total_delivery_status, 
    cc.num_cus_return, 
    tcus.total_customer, 
    tc.actual_sales, 
    tc.actual_profit
  

FROM 
    delivery_status_4_cte ds 
JOIN
    total_cte tc ON ds.year = tc.year AND ds.month = tc.month AND ds.store_key = tc.store_key
JOIN 
    cus_cte cc ON ds.year = cc.year AND ds.month = cc.month AND ds.store_key = cc.store_key
JOIN total_cus tcus ON tcus.year  = cc.year AND tcus.month  = cc.month AND tcus.store_key = cc.store_key

""")

StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 135, Finished, Available)

In [134]:
fact_monthly_kpi = fact_monthly_kpi.withColumn("date_key", col("date_key").cast("int"))

StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 136, Finished, Available)

In [ ]:
fact_monthly_kpi.dtypes

StatementMeta(, , , Cancelled, )

## fact_weekly_delivery_performance

In [ ]:
fact_weekly_delivery_performance = spark.sql("""
WITH cte AS (
    SELECT
        d.year, 
        d.month, 
        o.order_time_key,

        o.territory_key,
        o.order_date_key,

        SUM(CASE WHEN o.delivery_status_key = 3 THEN 1 ELSE 0 END) AS total_cancellation,
        COUNT(o.order_id) AS total_orders,
        SUM(CASE WHEN o.delivery_status_key = 2 THEN 1 ELSE 0 END) AS late_orders,
        SUM(CASE WHEN o.delivery_status_key = 4 THEN 1 ELSE 0 END) AS on_time_orders, 
        SUM(CASE WHEN o.delivery_status_key = 1 THEN 1 ELSE 0 END) AS advanced_orders 
    FROM
        (SELECT order_date_key, order_time_key, delivery_status_key, order_id, territory_key
         FROM orders
         GROUP BY order_date_key, order_time_key, delivery_status_key, order_id, territory_key) o
    JOIN
        dim_date d ON o.order_date_key = d.date_key
    GROUP BY
        o.territory_key, d.year, d.month, o.order_date_key, o.order_time_key
)
SELECT 
    order_date_key,
    order_time_key,
    territory_key,
    total_cancellation,
    on_time_orders, 
    late_orders,
    advanced_orders,
    total_orders
FROM cte;


""")

StatementMeta(, , , Cancelled, )

In [ ]:
fact_weekly_delivery_performance.dtypes

StatementMeta(, , , Cancelled, )

In [ ]:
fact_weekly_delivery_performance = fact_weekly_delivery_performance.withColumn("order_date_key", col("order_date_key").cast("int"))\
.withColumn("order_time_key", col("order_time_key").cast("int"))
fact_weekly_delivery_performance.dtypes

StatementMeta(, , , Cancelled, )

# Load data

In [ ]:
fact_sales.write.mode("append").format("delta").saveAsTable("fact_sales")

StatementMeta(, , , Cancelled, )

In [135]:
fact_monthly_kpi.write.mode("append").format("delta").saveAsTable("fact_monthly_kpi")

StatementMeta(, 5134a77f-80c0-44f8-985f-081f01432017, 137, Finished, Available)

In [ ]:
fact_weekly_delivery_performance.write.mode("append").format("delta").saveAsTable("fact_weekly_delivery_performance")

StatementMeta(, , , Cancelled, )